<h2>Gom nhóm dữ liệu</h2>
Đầu tiên, chúng ta sẽ nạp thư viện dplyr và thực hiện các bước như ở phần trước.
Chúng ta sẽ làm việc với file log từ ngày 8 tháng bảy, 2014 (chứa thông tin của gần 225,000 package được download). Các bạn có thể download ở link này: (http://cran-logs.rstudio.com/2014/2014-07-08.csv.gz)

Giả sử path2csv là đường dẫn đến file bạn vừa download ở trên (đã được giải nén thành file .csv). Ta sử dụng hàm read.csv() để load dữ liệu như sau:

In [ ]:
library(dplyr)

In [2]:
# tạo biến cran thuộc kiểu dữ liệu data frame tbl
mydf <- read.csv(path2csv, as.is = TRUE)

cran <- tbl_df(mydf)
head(cran)

date,time,size,r_version,r_arch,r_os,package,version,country,ip_id
2014-07-08,00:54:41,80589,3.1.0,x86_64,mingw32,htmltools,0.2.4,US,1
2014-07-08,00:59:53,321767,3.1.0,x86_64,mingw32,tseries,0.10-32,US,2
2014-07-08,00:47:13,748063,3.1.0,x86_64,linux-gnu,party,1.0-15,US,3
2014-07-08,00:48:05,606104,3.1.0,x86_64,linux-gnu,Hmisc,3.14-4,US,3
2014-07-08,00:46:50,79825,3.0.2,x86_64,linux-gnu,digest,0.6.4,CA,4
2014-07-08,00:48:04,77681,3.1.0,x86_64,linux-gnu,randomForest,4.6-7,US,3


Tiếp theo ta sẽ gom nhóm biến cran theo thuộc tính package và lưu kết quả vào biến mới gọi là by_package.

In [3]:
by_package <- group_by(cran, package)
print(head(by_package))

# A tibble: 6 x 10
# Groups:   package [6]
  date       time     size r_version r_arch r_os  package version country ip_id
  <chr>      <chr>   <int> <chr>     <chr>  <chr> <chr>   <chr>   <chr>   <int>
1 2014-07-08 00:54…  80589 3.1.0     x86_64 ming… htmlto… 0.2.4   US          1
2 2014-07-08 00:59… 321767 3.1.0     x86_64 ming… tseries 0.10-32 US          2
3 2014-07-08 00:47… 748063 3.1.0     x86_64 linu… party   1.0-15  US          3
4 2014-07-08 00:48… 606104 3.1.0     x86_64 linu… Hmisc   3.14-4  US          3
5 2014-07-08 00:46…  79825 3.0.2     x86_64 linu… digest  0.6.4   CA          4
6 2014-07-08 00:48…  77681 3.1.0     x86_64 linu… random… 4.6-7   US          3


Ở dòng trên cùng, ta thấy "Groups: package" cho chúng ta biết tbl này đã được gom nhóm theo thuộc tính package. Các phép toán áp dụng sau này của chúng ta sẽ tác động lên từng nhóm đã gom được này.

Khi áp dụng hàm mean(size) tlb_df thông qua hàm summarize(). Ta nhận được một giá trị mean duy nhất thể hiện giá trị trung bình của cột thuộc tính này. Nhưng nếu ta muốn biết giá trị download trung bình của từng package thì sao.


In [4]:
head(summarize(by_package, mean(size)))

package,mean(size)
A3,62194.96
ABCExtremes,22904.33
ABCoptim,17807.25
ABCp2,30473.33
ACCLMA,33375.53
ACD,99055.29


Thay vì trả về một giá trị, hàm summarize() bây giờ trả về giá trị download trung bình của từng package mà chúng ta đã gom nhóm trước đó. Ta phân tích dòng lệnh sau:


In [5]:
pack_sum <- summarize(by_package,
                      count = n(),
                      unique = n_distinct(ip_id),
                      countries = n_distinct(country),
                      avg_bytes = mean(size))

In [6]:
# in pack_sum ra màn hình
head(pack_sum)

package,count,unique,countries,avg_bytes
A3,25,24,10,62194.96
ABCExtremes,18,17,9,22904.33
ABCoptim,16,15,9,17807.25
ABCp2,18,17,10,30473.33
ACCLMA,15,14,9,33375.53
ACD,17,16,10,99055.29


Cột thuộc tính 'count' được tạo ra bởi hàm n(), chứa số lượng downloads của từng package. Cột thuộc tính 'unique' được tạo bởi hàm n_distinct(ip_id), chứa số lượng download từ địa chỉ ip người dùng. Cột thuộc tính 'countries' được tạo bởi hàm n_distinct(country), chứa số lượng các quốc gia download package này. Cuối cùng là cột thuộc tính 'avg_bytes' được tạo bởi hàm mean(size), chứa giá trị download trung bình của từng package.

Điều quan trọng là ta hiểu được ý nghĩa của từng cột thuộc tính của pack_sum và chúng được tạo ra như thế nào. Bây giờ chúng ta đã biết cách summarize() dữ liệu cho từng nhóm package. Tiếp theo, chúng ta thử làm một vài ví dụ khác để ôn lại những gì đã học.

Một cách tự nhiên, chúng ta sẽ muốn biết những package nào phổ biến nhất. Chúng ta bắt đầu bằng việc tách ra top 1% nhóm các package có số lượng download nhiều nhất dựa trên cột thuộc tính "count". Ta sẽ dùng bách phân vị 99% (quartile) để xác định giá trị của nhóm 99%. Từ đó, ta sẽ biết được ngưỡng để lấy 1% nhóm còn lại.


In [7]:
quantile(pack_sum$count, probs = 0.99)

99% 
679.56

Bây giờ, ta sẽ dùng ngưỡng 679 để tách ra nhóm 1%. Sử dụng hàm filter() giúp chọn ra các dòng dữ liệu có 'count' lớn hơn 679. Sau đó, lưu kết quả này vào biến top_counts.


In [8]:
top_counts <- filter(pack_sum, count > 679)
print(top_counts)

# A tibble: 61 x 5
   package      count unique countries avg_bytes
   <chr>        <int>  <int>     <int>     <dbl>
 1 DBI           2599    492        48   206933.
 2 Formula        852    777        65   155742.
 3 Hmisc          954    812        69  1367676.
 4 LPCM          2335     17        10   526814.
 5 MASS           834    698        66   981152.
 6 Matrix         932    801        66  3220134.
 7 RColorBrewer  1890   1584        79    22764.
 8 RCurl         1504   1207        73  1903505.
 9 RJDBC          809    107        28    18715.
10 RJSONIO        751    585        60  1208104.
# ... with 51 more rows


Chỉ có 61 package trong top 1% vì vậy ta muốn xem toàn bộ kết quả. Trong khi dplyr chỉ cho chúng ta xem 10 dòng dữ liệu đầu tiên, chúng ta có thể dùng hàm View() để xem toàn bộ dữ liệu.

Chúng ta muốn các package có giá trị cao nhất xuất hiện đầu bảng, nghĩa là chúng ta sẽ sắp xếp giảm dần cột thuộc tính count.

In [9]:
top_counts_sorted <- arrange(top_counts, desc(count))
print(top_counts_sorted)

# A tibble: 61 x 5
   package  count unique countries avg_bytes
   <chr>    <int>  <int>     <int>     <dbl>
 1 ggplot2   4602   1680        81  2427716.
 2 Rcpp      3195   2044        84  2512100.
 3 plyr      2908   1754        81   799123.
 4 rJava     2773    963        70   633522.
 5 DBI       2599    492        48   206933.
 6 LPCM      2335     17        10   526814.
 7 stringr   2267   1948        82    65277.
 8 digest    2210   1894        83   120549.
 9 reshape2  2032   1652        76   330128.
10 foreach   1984    485        53   358070.
# ... with 51 more rows


Theo kết quả thống kê trên, ta thấy dẫn đầu là package ggplot2 với 4602 lượt download, theo đó là Rcpp, plyr, rJava, ... Cuối cùng ta thấy swirl package đứng thứ 43 với 820 lượt download. Thật thú vị.
<h2>Kĩ thuật chaining</h2>
Kĩ thuật chaining cho phép chúng ta gọi các hàm nối tiếp nhau nhờ vậy mà code của chúng ta dễ đọc hơn trong khi vẫn cho ra cùng kết quả ban đầu. Cụ thể, ta sẽ tính toán lại các thao tác trên với kĩ thuật chaining.


In [10]:
cran %>%
group_by(package) %>%
summarize(count = n(),
        unique = n_distinct(ip_id),
        countries = n_distinct(country),
        avg_bytes = mean(size)
) %>%
filter(count > 679) %>%
arrange(desc(count)) %>%
head

package,count,unique,countries,avg_bytes
ggplot2,4602,1680,81,2427716.1
Rcpp,3195,2044,84,2512100.4
plyr,2908,1754,81,799122.8
rJava,2773,963,70,633522.3
DBI,2599,492,48,206933.3
LPCM,2335,17,10,526814.2


Qua bài viết này, ta đã học được cách gom nhóm và sử dụng kĩ thuật chaining với dplyr. Kết hợp với những gì đã học ở phần trước ta đã có thêm các công cụ mạnh mẽ, chính xác, và hiệu quả. Chào mừng đến với thế giới của dplyr.

<strong>Nguồn tham khảo:</strong> <a href="http://swirlstats.com/" target="_blank" rel="noopener">http://swirlstats.com/</a>